In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from PIL import Image
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

In [2]:
class Generator(keras.Model):
  def __init__(self):
    super(Generator,self).__init__()
    #encoder
    self.fc_layer_1 = layers.Dense(3*3*512)
    self.conv_1 = layers.Conv2DTranspose(256,3,3,'valid')

    self.bn_1 = layers.BatchNormalization()       
    self.conv_2 = layers.Conv2DTranspose(128,5,2,'valid')
    self.bn_2 = layers.BatchNormalization()     
    self.conv_3 = layers.Conv2DTranspose(3,4,3,'valid')

  def call(self, inputs, training=None):
    x = self.fc_layer_1(inputs)
    x = tf.reshape(x,[-1,3,3,512])
    x = tf.nn.relu(x)
    x = self.bn_1(self.conv_1(x),training=training)
    x = self.bn_2(self.conv_2(x),training=training)
    x = self.conv_3(x)
    x = tf.tanh(x)
    return x

In [3]:
class Discriminator(keras.Model):
  def __init__(self):
    super(Discriminator,self).__init__()

    self.conv_1 = layers.Conv2D(64,5,3,'valid')
    self.conv_2 = layers.Conv2D(128,5,3,'valid')
    self.bn_1 = layers.BatchNormalization()
    self.conv_3 = layers.Conv2D(256,5,3,'valid')    
    self.bn_2 = layers.BatchNormalization()
    self.flatten = layers.Flatten()
    self.fc_layer = layers.Dense(1)

  
  def call(self, inputs, training=None):
    x = tf.nn.leaky_relu(self.conv_1(inputs))    
    x = tf.nn.leaky_relu(self.bn_1(self.conv_2(x),training=training))    
    x = tf.nn.leaky_relu(self.bn_2(self.conv_3(x),training=training))  
    x = self.flatten(x)
    x = self.fc_layer(x)
    return x

In [4]:
g = Generator()
d = Discriminator()

In [5]:
x = tf.random.normal([1,64,64,3])
z = tf.random.normal([1,100])
prob = g(x)
print(prob)
out = d(x)
print(out.shape)

tf.Tensor(
[[[[ 4.67259786e-04  8.10428639e-04 -8.74764344e-04]
   [ 7.20210373e-04 -2.27877521e-04 -7.39809533e-04]
   [ 2.60381959e-04  6.48580317e-04  1.01465278e-03]
   ...
   [-1.00771338e-03 -3.51856608e-04 -4.18512500e-05]
   [ 6.51708338e-04  2.77312531e-04 -6.39891950e-04]
   [ 3.21032567e-04  3.35694494e-04  1.06416515e-03]]

  [[-3.10898351e-04 -1.08868153e-04 -6.12522344e-05]
   [-7.17712042e-04 -3.87944572e-04  6.48859015e-04]
   [-6.01540261e-04  2.50349869e-04  4.48149629e-04]
   ...
   [ 4.53540502e-04 -3.60390462e-04 -1.34839749e-04]
   [-1.60834554e-03  5.61223947e-04  8.75865982e-04]
   [-5.95626596e-04  9.88714979e-04  8.88301933e-04]]

  [[ 1.67019828e-03 -3.38440615e-04 -2.26883043e-04]
   [ 9.74697177e-04 -3.77938384e-04  1.22128322e-03]
   [-3.51049166e-05  9.39873105e-04 -2.78816093e-04]
   ...
   [ 6.12009782e-04  1.02984509e-03  9.34415963e-04]
   [-3.04952773e-05 -1.00131554e-03 -1.52219654e-04]
   [ 1.23400587e-05  1.71280321e-04 -4.38817602e-04]]

  ...

 